In [6]:
import os
import webbrowser
import psutil

def open_chrome():
    webbrowser.open("https://www.google.com")

def open_calculator():
    os.system("calc")

def get_cpu_usage():
    return psutil.cpu_percent(interval=1)


# print("CPU Usage:", get_cpu_usage(), "%")

function_bank = {
    "open_chrome": {
        "function": open_chrome,
        "description": "Open/Start Google Chrome browser",
        "code": """
                   import webbrowser
                   def open_chrome():
                   webbrowser.open('https://www.google.com')"""
    },

    "open_calculator": {
        "function": open_calculator,
        "description": "Open/Start/Launch the system calculator application",
        "code": """def open_calculator():
                   import os
                   os.system('calc')"""
    },

    "get_cpu_usage": {
        "function": get_cpu_usage,
        "description": "Get/Retrieve/Check/Show CPU usage of the system",
        "code": """def get_cpu_usage():
                   import psutil
                   return psutil.cpu_percent(interval=1)"""
    }
}

In [117]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from function_registry import function_bank
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from groq import Groq
from sentence_transformers import SentenceTransformer
import torch

app = FastAPI()


class QueryRequest(BaseModel):
    query: str

class Retriever:
    def __init__(self):
        # self.model=SentenceTransformer(model, device="cpu")
        self.embedding_model=None
        self.documents=None
        self.vector_store=None
        self.chat_history=[]

    def create_embeddings(self):
        self.embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device": "cpu"})
        self.documents=[
            Document(page_content=func["description"],metadata={"name":name})
            for name, func in function_bank.items()
        ]
        self.vector_store = FAISS.from_documents(self.documents, self.embedding_model)

    def retrieve(self,user_query):
        docs=self.vector_store.similarity_search(user_query, k=1)
        if docs:
            return docs[0].metadata["name"]
        return None
    
    def execute(self,user_query):
        function=self.retrieve(user_query)
        
        if function and function in function_bank:
            function_bank[f"{function}"]["function"]()
            return {"Executed": function, "Result": "Program Launched"}
        raise HTTPException(status_code=404, detail="Function not found.")
    

    def update_chat_history(self,user_query,model_response):
        self.chat_history.append({"user": user_query, "model": model_response})
        if (len(self.chat_history)>7):
            self.chat_history.pop(0)

    def retrieve_contextual_response(self,code_to_improve):
        context = "\n".join([f"User: {entry['user']}\nModel: {entry['model']}" for entry in self.chat_history])
        prompt = f"Context:\n{context}\n\nNew Query: {code_to_improve}\nProvide an improved response based on past interactions."
        return prompt

    def custom_function(self,new_query):

        return 
    def Invocation(self,user_query):
        function_name=self.retrieve(user_query)

        if not function_name:
            raise HTTPException(status_code=404, detail="No matching function found.")
        
        code_to_improve = f"""{function_bank[function_name]["code"]}"""
        prompt = "Ensure proper imports, error handling, and modularity."

# Initialize the client
        client = Groq(api_key="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")
        code_to_improve=self.retrieve_contextual_response(code_to_improve)
        # Create a chat completion request
        completion = client.chat.completions.create(
            model="llama-3.3-70b-specdec",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": code_to_improve}
            ],
            temperature=1,
            max_completion_tokens=1024,
            top_p=1,
            stream=True,
            stop=None,
        )
        response = ""
        for chunk in completion:
            if chunk.choices[0].delta.content:
                response += chunk.choices[0].delta.content
                # print(chunk.choices[0].delta.content, end="")  # Print as it streams

        self.update_chat_history(user_query, response)
        return {"function": function_name, "improved_code": response}
        # return response


        

        
 # Executes get_cpu_usage()


In [114]:
R=Retriever()

In [108]:
@app.post("/execute")
async def execute_function(request: QueryRequest):
    """API endpoint to execute a function."""
    return R.execute(request.query)


@app.post("/improve_code")
async def improve_code(request: QueryRequest):
    """API endpoint to retrieve and enhance function code."""
    return await R.invocation(request.query)


In [115]:
R.create_embeddings()

In [116]:
query = "Launch Web browser"
improved_code = R.execute(query)
print(improved_code)

{'Executed': 'open_chrome', 'Result': 'Program Launched'}


In [100]:
output=R.Invocation("Open calc")

In [102]:
output

{'function': 'open_calculator',
 'improved_code': '**Improved Response: Enhanced Calculator Function**\n=====================================================\n\nThe provided code snippet attempts to launch the system\'s default calculator application. However, it lacks proper error handling, modularity, and is limited to Windows systems. Here\'s an improved version of the `open_calculator` function:\n\n```python\nimport os\nimport sys\nimport platform\n\ndef open_calculator():\n    """\n    Launches the system\'s default calculator application.\n\n    Raises:\n        OSError: If the calculator application cannot be found.\n        RuntimeError: If the operating system is not supported.\n    """\n    try:\n        # Determine the calculator command based on the operating system\n        if platform.system() == \'Windows\':\n            calculator_command = \'calc\'\n        elif platform.system() == \'Darwin\':  # macOS\n            calculator_command = \'open -a Calculator\'\n        

In [103]:
output=output['improved_code']

In [55]:
R.chat_history

[{'user': 'Launch my Chrome',
  'model': '### Improved Code with Error Handling and Modularity\n\nHere\'s a refactored version of your code with proper imports, error handling, and modularity:\n\n```python\nimport webbrowser\n\ndef open_browser(url):\n    """\n    Opens a URL in the default browser.\n\n    Args:\n        url (str): The URL to open.\n\n    Returns:\n        bool: True if successful, False otherwise.\n    """\n    try:\n        webbrowser.open(url)\n        return True\n    except webbrowser.Error as e:\n        print(f"Error opening browser: {e}")\n        return False\n\ndef main():\n    url = \'https://www.google.com\'\n    if open_browser(url):\n        print(f"Opened {url} successfully.")\n    else:\n        print(f"Failed to open {url}.")\n\nif __name__ == "__main__":\n    main()\n```\n\n### Changes Made\n\n1.  **Modularized the code**: The `open_browser` function now takes a URL as an argument, making it reusable.\n2.  **Added error handling**: The `open_browser` 

In [48]:
def retrieve_contextual_response(user_query):
    context = "\n".join([f"User: {entry['user']}\nModel: {entry['model']}" for entry in R.chat_history])
    prompt = f"Context:\n{context}\n\nNew Query: {user_query}\nProvide an improved response based on past interactions."
    return prompt


In [50]:
output=retrieve_contextual_response("Check cpu usage")

In [8]:
function_name=R.retrieve("Launch Google chrome")
print(function_bank[function_name]["code"])


                   import webbrowser
                   def open_chrome():
                   webbrowser.open('https://www.google.com')


In [104]:
for line in output.splitlines():
    print(line)

**Improved Response: Enhanced Calculator Function**

The provided code snippet attempts to launch the system's default calculator application. However, it lacks proper error handling, modularity, and is limited to Windows systems. Here's an improved version of the `open_calculator` function:

```python
import os
import sys
import platform

def open_calculator():
    """
    Launches the system's default calculator application.

    Raises:
        OSError: If the calculator application cannot be found.
        RuntimeError: If the operating system is not supported.
    """
    try:
        # Determine the calculator command based on the operating system
        if platform.system() == 'Windows':
            calculator_command = 'calc'
        elif platform.system() == 'Darwin':  # macOS
            calculator_command = 'open -a Calculator'
        elif platform.system() == 'Linux':
            # Try to launch the default calculator for Linux distributions
            calculator_command

API Integration for Invocation

In [9]:
from groq import Groq

# Define the function to improve (retrieved from elsewhere)
code_to_improve = f"""{function_bank[function_name]["code"]}"""


# Define the improvement prompt separately
prompt = "Ensure proper imports, error handling, and modularity."

# Initialize the client
client = Groq(api_key="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")

# Create a chat completion request
completion = client.chat.completions.create(
    model="llama-3.3-70b-specdec",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": code_to_improve}
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

# Stream and print the improved code
for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


### Improved Code

Here's a more robust and maintainable version of your code. This version includes proper imports, error handling, and modularity.

```python
# Import the required module
import webbrowser

# Define a function to open a URL in the default browser
def open_url(url):
    """
    Open a URL in the default browser.
    
    Args:
        url (str): The URL to be opened.
    
    Returns:
        None
    """
    try:
        # Attempt to open the URL
        webbrowser.open(url)
    except Exception as e:
        # Handle any exceptions that occur
        print(f"An error occurred: {e}")

# Define a function to open Google in Chrome
def open_google():
    """
    Open Google in the default browser.
    
    Returns:
        None
    """
    url = 'https://www.google.com'
    open_url(url)

# Example usage:
if __name__ == "__main__":
    open_google()
```

### Changes Made

1. **Added a docstring**: The `open_url` function now has a docstring that explains what the functio